# Problema de Otimização: Aquisição de Aeronaves

A **Empresa de Viação Aérea Brasileira** estuda a compra de três tipos de aviões:

- **Boeing 717** (curta distância)  
- **Boeing 737-500** (média distância)  
- **MD-11** (longa distância)  

Os dados de custo, receita e pilotos aptos estão na tabela:

| Avião            | Custo (milhões \$) | Receita (milhões \$) | Pilotos aptos |
|------------------|---------------------|-----------------------|---------------|
| Boeing 717       | 5.1                 | 330                   | 30            |
| Boeing 737-500   | 3.6                 | 300                   | 20            |
| MD-11            | 6.8                 | 420                   | 10            |

**Outras restrições:**
- Verba disponível: **220 milhões de \$**.  
- Cada aeronave precisa de **2 pilotos**.  
- Pilotos do MD-11 podem pilotar **qualquer aeronave**; os demais só pilotam o modelo em que são habilitados.  
- Capacidade de manutenção: **40 Boeings 717 equivalentes**.  
  - Boeing 717 = 1 unidade  
  - Boeing 737-500 = 3/4 de unidade  
  - MD-11 = 5/3 de unidade  

---

## Modelagem Matemática

### Variáveis de decisão
- $x_1$ = número de **Boeings 717** comprados.  
- $x_2$ = número de **Boeings 737-500** comprados.  
- $x_3$ = número de **MD-11** comprados.  

Para modelar a **alocação de pilotos**:  
- $u_1$ = pilotos do pool 717 usados em 717.  
- $u_2$ = pilotos do pool 737 usados em 737.  
- $v_{31}$ = pilotos do pool MD-11 usados em 717.  
- $v_{32}$ = pilotos do pool MD-11 usados em 737.  
- $v_{33}$ = pilotos do pool MD-11 usados em MD-11.  


---

### Função Objetivo
Maximizar a receita total obtida pelas aeronaves:

$$
\max Z = 330x_1 + 300x_2 + 420x_3
$$

---

### Restrições

**1. Orçamento disponível:**

$$
5.1x_1 + 3.6x_2 + 6.8x_3 \;\leq\; 220
$$

---

**2. Pilotos necessários por aeronave:**

$$
u_1 + v_{31} = 2x_1
$$

$$
u_2 + v_{32} = 2x_2
$$

$$
v_{33} = 2x_3
$$

---

**3. Limite de cada pool de pilotos:**

$$
u_1 \leq 30
$$

$$
u_2 \leq 20
$$

$$
v_{31} + v_{32} + v_{33} \leq 10
$$

---

**4. Capacidade de manutenção:**

$$
x_1 + \tfrac{3}{4}x_2 + \tfrac{5}{3}x_3 \;\leq\; 40
$$

---

**5. Domínio das variáveis:**

$$
x_j, u_1, u_2, v_{31}, v_{32}, v_{33} \in \mathbb{Z}_{\geq 0}
$$

---

## Observações

- As variáveis $u_1$, $u_2$, $v_{31}$, $v_{32}$, $v_{33}$ foram incluídas para respeitar a **habilitação específica dos pilotos**.  
- Se considerássemos apenas o total de pilotos, poderíamos perder a restrição de compatibilidade.  
- A restrição de manutenção foi convertida para "equivalente a Boeing 717", conforme o enunciado.  
- O modelo busca **maximizar receita**, sujeito a orçamento, pilotos e manutenção.

---


In [ ]:
import pulp

# Parâmetros
c = {1: 5.1, 2: 3.6, 3: 6.8}      # custos (milhões)
r = {1: 330, 2: 300, 3: 420}      # receitas (unidades conforme enunciado)
B = 220                           # verba disponível (milhões)
P = {1: 30, 2: 20, 3: 10}         # pilotos aptos por pool
M_max = 40                        # capacidade manutenção (equivalente a 717)

# Modelo
model = pulp.LpProblem("Compra_Aeronaves", pulp.LpMaximize)

# Variáveis de decisão: quantas aeronaves de cada tipo comprar
x = {j: pulp.LpVariable(f"x_{j}", lowBound=0, cat="Integer") for j in [1,2,3]}

# Variáveis de alocação de pilotos
u1 = pulp.LpVariable("u1_717_pool", lowBound=0, cat="Integer")  # pilotos pool 717 usados em 717
u2 = pulp.LpVariable("u2_737_pool",  lowBound=0, cat="Integer")  # pilotos pool 737 usados em 737
v31 = pulp.LpVariable("v31_MD11_to_717", lowBound=0, cat="Integer")
v32 = pulp.LpVariable("v32_MD11_to_737", lowBound=0, cat="Integer")
v33 = pulp.LpVariable("v33_MD11_to_MD11", lowBound=0, cat="Integer")

# Objetivo: maximizar receita
model += r[1]*x[1] + r[2]*x[2] + r[3]*x[3], "Maximize_Revenue"

# Restrição de orçamento
model += c[1]*x[1] + c[2]*x[2] + c[3]*x[3] <= B, "Budget"

# Restricoes de pilotos (balanceamento de pilotos por aeronave)
model += u1 + v31 == 2 * x[1], "Pilots_for_717"
model += u2 + v32 == 2 * x[2], "Pilots_for_737"
model += v33 == 2 * x[3], "Pilots_for_MD11"

# Limites dos pools de pilotos
model += u1 <= P[1], "Pool717_capacity"
model += u2 <= P[2], "Pool737_capacity"
model += v31 + v32 + v33 <= P[3], "PoolMD11_capacity"

# Restricao de manutencao (equivalente a 717)
model += 1.0 * x[1] + (3.0/4.0) * x[2] + (5.0/3.0) * x[3] <= M_max, "Maintenance_capacity"

# Resolver
model.solve()

# Resultados
print("Status:", pulp.LpStatus[model.status])
print("Objective (receita total):", pulp.value(model.objective))
for j in [1,2,3]:
    print(f"x_{j} (nº de aeronaves tipo {j}):", int(pulp.value(x[j])))

print("\nAlocacao pilotos:")
print("u1 (pool 717 usados para 717):", int(pulp.value(u1)))
print("u2 (pool 737 usados para 737):", int(pulp.value(u2)))
print("v31 (MD11 -> 717):", int(pulp.value(v31)))
print("v32 (MD11 -> 737):", int(pulp.value(v32)))
print("v33 (MD11 -> MD11):", int(pulp.value(v33)))


Status: Optimal
Objective (receita total): 10050.0
x_1 (nº de aeronaves tipo 1): 15
x_2 (nº de aeronaves tipo 2): 10
x_3 (nº de aeronaves tipo 3): 5

Alocacao pilotos:
u1 (pool 717 usados para 717): 30
u2 (pool 737 usados para 737): 20
v31 (MD11 -> 717): 0
v32 (MD11 -> 737): 0
v33 (MD11 -> MD11): 10
